# Downscaled DTR

This notebook is for exploring downscaled daily temperature range data. We are specifically interested in DTR from GFDL-ESM4, because of the physically implausible tasmin values derived from tasmax - dtr, but other models exhibit some suspicious cold values as well.

In [ ]:
# Set ERA5_DIR, CMIP6_DIR, and ADJ_DIR environment variables to your paths
# export ERA5_DIR=/path/to/cmip6_4km_downscaling/era5_zarr
# export CMIP6_DIR=/path/to/cmip6_4km_downscaling/cmip6_zarr
# export ADJ_DIR=/path/to/cmip6_4km_downscaling/adjusted

# Pass parameters via command line arguments like so, for example:
# papermill downscaled_dtr.ipynb output.ipynb -p models 'EC-Earth3-Veg' -p scenarios 'historical ssp370 ssp585'
models = "EC-Earth3-Veg"
scenarios = "historical ssp370 ssp585"

In [ ]:
import os
import xarray as xr
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import Markdown

import warnings
warnings.filterwarnings('ignore')

models = models.split(" ")
scenarios = scenarios.split(" ")

ref_dir = Path(
    os.getenv("ERA5_DIR", "/beegfs/CMIP6/crstephenson/cmip6_4km_downscaling/era5_zarr")
)

cmip6_dir = Path(
    os.getenv("CMIP6_DIR", "/beegfs/CMIP6/crstephenson/cmip6_4km_downscaling/cmip6_zarr")
)

adj_dir = Path(
    os.getenv("ADJ_DIR", "/beegfs/CMIP6/crstephenson/cmip6_4km_downscaling/adjusted")
)

In [ ]:
def plot_dtr(era5_ds, ds, model, scenario):
    dtr = ds["dtr"]
    dtr.attrs["units"] = "°C"

    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    # Compute min values for both datasets
    dtr_min = dtr.min("time")
    era5_min = era5_ds["dtr"].min("time")

    # Find global min and max for consistent color scale
    # vmin = min(float(dtr_min.min().values), float(era5_min.min().values))
    # vmax = max(float(dtr_min.max().values), float(era5_min.max().values))

    # Plot minimum DTR from downscaled data
    dtr_min.plot(ax=ax[0], cmap="coolwarm", x='x', y='y')
    ax[0].set_title(f"Minimum DTR ({model} Downscaled)")

    # Plot minimum DTR from ERA5 reference data
    era5_min.plot(ax=ax[1], cmap="coolwarm")
    ax[1].set_title("Minimum DTR (ERA5)")

    plt.tight_layout()
    plt.show()

    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    # Compute mean values for both datasets
    dtr_mean = dtr.mean("time")
    era5_mean = era5_ds["dtr"].mean("time")

    # Find global min and max for consistent color scale
    vmin = min(float(dtr_mean.min().values), float(era5_mean.min().values))
    vmax = max(float(dtr_mean.max().values), float(era5_mean.max().values))

    # Plot mean DTR from downscaled data
    dtr_mean.plot(ax=ax[0], cmap="coolwarm", x='x', y='y', vmin=vmin, vmax=vmax)
    ax[0].set_title("Mean DTR (GFDL-ESM4 Downscaled)")

    # Plot mean DTR from ERA5 reference data
    era5_mean.plot(ax=ax[1], cmap="coolwarm", vmin=vmin, vmax=vmax)
    ax[1].set_title("Mean DTR (ERA5)")

    plt.tight_layout()
    plt.show()

    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    # Compute max values for both datasets
    dtr_max = dtr.max("time")
    era5_max = era5_ds["dtr"].max("time")

    # Find global min and max for consistent color scale
    vmin = min(float(dtr_max.min().values), float(era5_max.min().values))
    vmax = max(float(dtr_max.max().values), float(era5_max.max().values))

    # Plot maximum DTR from downscaled data
    dtr_max.plot(ax=ax[0], cmap="coolwarm", x='x', y='y', vmin=vmin, vmax=vmax)
    ax[0].set_title("Maximum DTR (GFDL-ESM4 Downscaled)")

    # Plot maximum DTR from ERA5 reference data
    era5_max.plot(ax=ax[1], cmap="coolwarm", vmin=vmin, vmax=vmax)
    ax[1].set_title("Maximum DTR (ERA5)")

    plt.tight_layout()
    plt.show()

    era5_ds["dtr"].max().values

    fig, ax = plt.subplots(1, 2, figsize=(14, 6))

    # Compute max values for both datasets
    dtr_masked = dtr.where(dtr <= 70)
    dtr_max = dtr_masked.max("time")
    era5_max = era5_ds["dtr"].max("time")

    # Find global min and max for consistent color scale
    vmin = min(float(dtr_max.min().values), float(era5_max.min().values))
    vmax = max(float(dtr_max.max().values), float(era5_max.max().values))

    # Plot maximum DTR from downscaled data
    dtr_max.plot(ax=ax[0], cmap="coolwarm", x='x', y='y', vmin=vmin, vmax=vmax)
    ax[0].set_title("Maximum DTR (GFDL-ESM4 Downscaled), DTR restricted to 70°C")

    # Plot maximum DTR from ERA5 reference data
    era5_max.plot(ax=ax[1], cmap="coolwarm", vmin=vmin, vmax=vmax)
    ax[1].set_title("Maximum DTR (ERA5) (maximum ERA5 DTR is 66°C)")

    plt.tight_layout()
    plt.show()

In [ ]:
era5_ds = xr.open_dataset(ref_dir.joinpath("dtr_era5.zarr"))
for model in models:
    display(Markdown(f"# {model}"))
    for scenario in scenarios:
        zarr_store = adj_dir.joinpath(f"dtr_{model}_{scenario}_adjusted.zarr")
        ds = xr.open_zarr(zarr_store)
        display(Markdown(f"## {model}, {scenario}"))
        plot_dtr(era5_ds, ds, model, scenario)